In [35]:
import pandas as pd
import json
import multiprocessing
from multiprocessing import Pool

In [27]:
def get_data_and_header(nrows=None):
    if nrows:
        df = pd.read_csv('../Data/usa_00021.csv', nrows = nrows)
    else:
        df = pd.read_csv('../Data/usa_00021.csv')

    drop_list = ['DATANUM', 'SERIAL', 'CBSERIAL', 'PERNUM', 'RACED', 'BPLD', 'LANGUAGED', 'OCC', 'IND', 'HHWT']
    df.drop(drop_list, axis=1, inplace=True)
    header = list(df)
    return df, header


df, header = get_data_and_header(nrows = 100000)

In [28]:
def read_json(f_name):
    with open(f_name) as j:  
        codebook = json.load(j)
    return codebook

In [29]:
def is_skip_case(h):
    # these are basically numeric terms
    return h in ['INCOME', 'HHWT', 'FARMPROD', 'MORTAMT1', 'PROPINSR', 'RENTGRS', 'CONDOFEE', 'MOBLHOME'
                ,'COSTELEC', 'COSTGAS', 'COSTWATR', 'COSTFUEL', 'HHINCOME', 'VALUEH', 'PERWT',
                'MOMLOC', 'POPLOC', 'BIRTHYR', 'RACED', 'YRIMMIG', 'YRSUSA1', 'OCC', 'IND', 'INCTOT',
                'FTOTINC', 'INCSS', 'INCWELFR', 'INCRETIR', 'POVERTY', 'OCCSCORE', 'SEI', 'HWSEI',
                'PRESGL', 'PRENT', 'MOVEDIN', 'RIDERS', 'TRANTIME', 'DEPARTS', 'ROOMS']


# It works well with variables defined within the kernel
# It fails when we call other functions inside the process
# But it won't error out, it just returns a None Type 
def transform_row(row):
    new_row = row[:]
    exceptions = {}
    for idx, each in enumerate(row):
        
        h = header[idx]

        if h == 'YEAR':
            new_row[idx] = each
            continue

        key = str(int(each))

        # start the special cases here         
        if is_skip_case(h):
            new_row[idx] = int(key) 
            continue
            
        try:
            g = codes[h][key]
        except:
            print (h, key)
            break
#             new_row[idx] = codes[h][key]

    return new_row

def magic_loop(rows):
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows= pool.map(transform_row, rows)
    pool.close() 
    pool.join()
    df = pd.DataFrame(transformed_rows, columns = header)
    return df

In [32]:
codes = read_json('../Data/codebook.json')

In [33]:
rows = df.as_matrix()

In [36]:
transformed_df = magic_loop(rows)

In [ ]:
other_categorical = ['MOVEDIN','MOMLOC','POPLOC','RIDERS']

In [ ]:
year_terms = ['BIRTHYR',  'YRIMMIG', 'YRSUSA1']

In [40]:
terms_to_scale = ['TRANTIME', 'DEPARTS', 'ROOMS']

In [45]:
scale_by_inflation = ['INCOME', 'FARMPROD', 'MORTAMT1', 'PROPINSR', 'RENTGRS' ,'CONDOFEE', 'MOBLHOME', 'COSTELEC',
                     'COSTGAS','COSTWATR','COSTFUEL','HHINCOME', 'VALUEH', 'INCTOT','FTOTINC',
                     'INCSS','INCWELFR', 'INCRETIR']


In [43]:
economic_terms = ['POVERTY', 'OCCSCORE', 'SEI', 'HWSEI','PRESGL', 'PRENT']

In [46]:
scale_by_inflation

['INCOME',
 'FARMPROD',
 'MORTAMT1',
 'PROPINSR',
 'RENTGRS',
 'CONDOFEE',
 'MOBLHOME',
 'COSTELEC',
 'COSTGAS',
 'COSTWATR',
 'COSTFUEL',
 'HHINCOME',
 'VALUEH',
 'INCTOT',
 'FTOTINC',
 'INCSS',
 'INCWELFR',
 'INCRETIR']